In [39]:
import pandas as pd
import numpy as np

In [40]:
# Step 1: Load the dataset
from google.colab import files
uploaded = files.upload()

Saving blogs.csv to blogs (1).csv


In [41]:
df = pd.read_csv('blogs.csv')

In [42]:
# Display first few rows
df.head()

,Data,Labels
0,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
1,Newsgroups: alt.atheism\nPath: cantaloupe.srv....,alt.atheism
2,Path: cantaloupe.srv.cs.cmu.edu!das-news.harva...,alt.atheism
3,Path: cantaloupe.srv.cs.cmu.edu!magnesium.club...,alt.atheism
4,Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53...,alt.atheism


In [43]:
# Rename columns for easier access
df.columns = ["Data", "Labels"]

In [44]:
# Drop any missing values
df.dropna(inplace=True)

In [45]:
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

df['Cleaned_Data'] = df['Data'].apply(preprocess_text)  # Function must be defined before use


In [47]:
from sklearn.model_selection import train_test_split

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [49]:
from sklearn.naive_bayes import MultinomialNB

In [50]:
from sklearn.metrics import accuracy_score, classification_report

In [51]:
from textblob import TextBlob

In [52]:
# Convert Text to Features using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
x = vectorizer.fit_transform(df['Cleaned_Data'])
y = df['Labels']

In [53]:
# Split Data
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=42)

In [55]:
# Train Naive Bayes Classifier
nb_model = MultinomialNB()
nb_model.fit(x_train, y_train)


MultinomialNB()

In [56]:
# Predictions
y_pred = nb_model.predict(x_test)

In [57]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8375
Classification Report:
                           precision    recall  f1-score   support

             alt.atheism       0.50      0.83      0.62        18
           comp.graphics       0.70      0.89      0.78        18
 comp.os.ms-windows.misc       0.91      0.91      0.91        22
comp.sys.ibm.pc.hardware       0.80      0.80      0.80        25
   comp.sys.mac.hardware       0.86      0.90      0.88        21
          comp.windows.x       0.95      0.84      0.89        25
            misc.forsale       1.00      0.78      0.88        18
               rec.autos       0.90      1.00      0.95        18
         rec.motorcycles       1.00      0.94      0.97        16
      rec.sport.baseball       0.84      0.89      0.86        18
        rec.sport.hockey       0.88      1.00      0.94        15
               sci.crypt       0.90      1.00      0.95        19
         sci.electronics       0.65      0.69      0.67        16
                 sci.med       0.8

In [58]:
# Sentiment Analysis using TextBlob
def get_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return "Positive"
    elif analysis.sentiment.polarity < 0:
        return "Negative"
    else:
        return "Neutral"

df['Sentiment'] = df['Data'].apply(get_sentiment)

In [59]:
# Display sentiment distribution
print(df['Sentiment'].value_counts())

Sentiment
Positive    1543
Negative     457
Name: count, dtype: int64


In [60]:
# Save results to a new CSV file
df.to_csv("blogs_with_sentiment.csv", index=False)